# Natural language to SQL

**Run in [Google Colab](https://colab.research.google.com/) For GPU.**

This model have  Mistral as a base and it has been fine-tuned to excel in SQL code generation.

In [3]:
# Install necessary libraries
!pip install google-auth-oauthlib google-auth-httplib2 googleapis
!apt-get install -qq curl gnupg2 software-properties-common apt-transport-https ca-certificates
!add-apt-repository -yes ppa:wandisco-ltd/ppa
!apt-get update -qq
!apt-get install -qq git-lfs

# Import necessary libraries
from google.colab import auth
from google.auth import default
import google.auth.transport.requests
from google.colab import userdata
import os

# Set the environment variable with the token content
# First, set token content
HF_TOKEN_CONTENT = "your_token_here" # Replace with the actual token value

# Check if the secret already exists
try:
    # Attempt to get the secret
    HF_TOKEN = userdata.get('HF_TOKEN')

except userdata.SecretNotFoundError:
    # Secret does not exist, create it
    # Use echo to write the token to a temporary file, then create the secret from the file
    os.environ['HF_TOKEN'] = HF_TOKEN_CONTENT
    !echo "$HF_TOKEN" > HF_TOKEN.txt
    userdata.set_from_file('HF_TOKEN', 'HF_TOKEN.txt')
    os.remove('HF_TOKEN.txt')  # Cleanup temporary file
    print("Secret 'HF_TOKEN' has been created.")

except userdata.NotebookAccessError:
    # Request access to the secret
    auth.authenticate_user()
    creds, _ = default()
    req = google.auth.transport.requests.Request()
    creds.refresh(req)
    os.environ['HF_TOKEN'] = HF_TOKEN_CONTENT
    !echo "$HF_TOKEN" > HF_TOKEN.txt
    userdata.set_from_file('HF_TOKEN', 'HF_TOKEN.txt')
    os.remove('HF_TOKEN.txt')  # Cleanup temporary file
    print("Access requested for secret 'HF_TOKEN'.")

# Finally, retrieve the token using `get`
HF_TOKEN = userdata.get('HF_TOKEN')

# Alternatively, use environment variables to store and access sensitive information.
# It's recommended to set the HF_TOKEN environment variable instead of storing it directly in the code.
# You can set it within your Colab environment settings or through the code:
# os.environ['HF_TOKEN'] = 'your_actual_token'
# HF_TOKEN = os.environ.get('HF_TOKEN')

ERROR: Could not find a version that satisfies the requirement googleapis (from versions: none)
ERROR: No matching distribution found for googleapis
Selecting previously unselected package apt-transport-https.
(Reading database ... 123634 files and directories currently installed.)
Preparing to unpack .../apt-transport-https_2.4.13_all.deb ...
Unpacking apt-transport-https (2.4.13) ...
Setting up apt-transport-https (2.4.13) ...
usage: add-apt-repository [-h] [-d] [-r] [-s] [-c COMPONENT] [-p POCKET] [-y] [-n] [-l]
                          [--dry-run] [-L] [-P PPA] [-C CLOUD] [-U URI]
                          [-S SOURCESLIST [SOURCESLIST ...]]
                          [line ...]
add-apt-repository: error: argument -y/--yes: ignored explicit argument 'es'
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)


AttributeError: module 'google.colab.userdata' has no attribute 'set_from_file'

In [ ]:
from google.colab import userdata
userdata.get('HF_TOKEN')

In [5]:
#Install the lastest versions of peft & transformers library recommended
#if you want to work with the most recent models
!pip install -q git+https://github.com/huggingface/peft.git
!pip install git+https://github.com/huggingface/accelerate.git
!pip install git+https://github.com/huggingface/transformers.git
!pip install bitsandbytes

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Cloning https://github.com/huggingface/accelerate.git to /tmp/pip-req-build-an0a9w_0
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/accelerate.git /tmp/pip-req-build-an0a9w_0
  Resolved https://github.com/huggingface/accelerate.git to commit d6d3e03cd4868b1f69d4f334ddd341e4d414c342
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for accelerate: filename=accelerate-1.2.0.dev0-py3-none-any.whl size=337050 sha256=249e63257145b24c5bad902dedd14fd4a526828df910baac876f5adf92fb40f7
  Stored in directory: /tmp/pip-ephem-wheel-cache-5xtyynqy/wheels/9c/a3/1e/47368f9b6575655fe9ee1b6350cfa7d4b0befe66a35f8a8365
Successfully built accelerate
  Attempting uninstall: accelerate
    Found existing installation: accelerate 1.2.1
 

In [7]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch
import accelerate

In [8]:
model_name = "defog/sqlcoder-7b"

We need to create the Quantization configuration to load the Model.

It is a large model and I want it to fit in a 16GB GPU, I'm going to use a 4 bits quantization.

If you want to learn more about quantization, refer to this article: [QLoRA: Training a Large Language Model on a 16GB GPU.](https://medium.com/towards-artificial-intelligence/qlora-training-a-large-language-model-on-a-16gb-gpu-00ea965667c1)

You can try to use this model in a 8 bit quantizations and check in you see any improvements in the results.

In [9]:
bnb_config = BitsAndBytesConfig(
  load_in_4bit=True,
  bnb_4bit_use_double_quant=True,
  bnb_4bit_quant_type="nf4",
  bnb_4bit_compute_dtype=torch.bfloat16
)


To load the model I pass to the AutoModelForCasualLM teh quantization configurations, and HuggingFace take care of all the hard work.

In [10]:
foundation_model = AutoModelForCausalLM.from_pretrained(model_name,
                    quantization_config=bnb_config,
                    device_map='auto',
                    use_cache = True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/619 [00:00<?, ?B/s]

pytorch_model.bin.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [11]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
eos_token_id = tokenizer.convert_tokens_to_ids(["```"])[0]

tokenizer_config.json:   0%|          | 0.00/915 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

This function wraps the call to *model.generate*

In [12]:
#this function returns the outputs from the model received, and inputs.
def get_outputs(model, inputs, max_new_tokens=400):
    outputs = model.generate(
        input_ids=inputs["input_ids"],
        attention_mask=inputs["attention_mask"],
        num_return_sequences=1,
        eos_token_id=eos_token_id,
        pad_token_id=eos_token_id,
        max_new_tokens=max_new_tokens,
        do_sample=False,
        num_beams=5
    )
    return outputs

# Prompt without Shots.
In this first PROMPT we are going to give Instructions to the model and pass the structure of the Database.

The instructions are significantly different from those we are passing to GPT-3.5-Turbo. This model is really well fine-tuned, but it is smaller than GPT-3.5.

We need to be more clear with the instructions, as it does not have the same capacity to understand our orders as GPT-3.5.

In [13]:
sp_nl2sql = """
    ### Instructions:
Your task is convert a question into a SQL query, given a SQL database schema.
Adhere to these rules:
- **Deliberately go through the question and database schema word by word** to appropriately answer the question

    ### Input
    Generate a SQL query that answers the question below.
    This query will run on a database whose schema is represented in this string:

    CREATE 3+ TABLES HERE

    ### Response
    Based on your instructions, here is the SQL query I have generated to answer the question
    `{question}`:
    ```sql3
    """

In [14]:
sp_nl2sql = sp_nl2sql.format(question="YOUR QUERY HERE")
print(sp_nl2sql)


    ### Instructions:
Your task is convert a question into a SQL query, given a SQL database schema.
Adhere to these rules:
- **Deliberately go through the question and database schema word by word** to appropriately answer the question

    ### Input
    Generate a SQL query that answers the question below.
    This query will run on a database whose schema is represented in this string:

    CREATE 3+ TABLES HERE

    ### Response
    Based on your instructions, here is the SQL query I have generated to answer the question
    `YOUR QUERY HERE`:
    ```sql3
    


In [17]:
input_sentences = tokenizer(sp_nl2sql, return_tensors="pt").to('cuda')
response = get_outputs(foundation_model, input_sentences, max_new_tokens=400)
SQL = tokenizer.batch_decode(response, skip_special_tokens=True)

In [ ]:
#Empty the cache in orde to do more calls without problems.
torch.cuda.empty_cache()

In [ ]:
print(SQL[0].split("```sql3")[-1].split("```")[0].split(";")[0].strip() + ";")

The SQL Order is correct.

#Prompt with shots OpenAI Style.
In this second prompt we are going to add some Shots with samples to see if our SQL style affects the model.

In [ ]:
sp_nl2sql2 = """
    ### Instructions:
Your task is convert a question into a SQL query, given a SQL database schema.
Adhere to these rules:
- **Deliberately go through the question and database schema word by word** to appropriately answer the question
- **Use the samples SQL In the ### Samples section to clearn more about teh Databases structure


    ### Input
    Generate a SQL query that answers the question below.
    This query will run on a database whose schema is represented in this string:

   YOUR TABLES HERE

    ### Response
    YOUR QERIES AND SAMPLE RESPONSES HERE

    `{question}`:
    ```sql3
    """


In [ ]:
sp_nl2sql2 = sp_nl2sql2.format(question="Return The name of the best paid employee")
(print(sp_nl2sql2))

In [ ]:
input_sentences = tokenizer(sp_nl2sql2, return_tensors="pt").to('cuda')
response = get_outputs(foundation_model, input_sentences, max_new_tokens=400)
SQL = tokenizer.batch_decode(response, skip_special_tokens=True)
torch.cuda.empty_cache()

In [ ]:
print(SQL[0].split("```sql3")[-1].split("```")[0].split(";")[0].strip() + ";")

The Order is really different from the one obtained with the first prompt.

The first difference is the format. But The SQL is realy more simple, at least it is my sensation.

#Prompt with Shots in Sample Style.

In this prompt, we will place the examples in a separate section, and in the instructions, we will instruct the model to pay attention to them in order to generate the SQL commands.

In [ ]:
sp_nl2sql3b = """
    ### Instructions:
Your task is convert a question into a SQL query, given a SQL database schema.
Adhere to these rules:
- **Deliberately go through the question and database schema word by word** to appropriately answer the question
- **Use the samples SQL In the ### Samples section to learn more about the Databases structure


    ### Input
    Generate a SQL query that answers the question below.
    This query will run on a database whose schema is represented in this string:

    YOUR TABLES HERE

    ### Samples

    YOUR SAMPLES HERE

    ### Response
    Based on your instructions, here is the SQL query I have generated to answer the question
    `{question}`:
    ```sql3
    """


In [ ]:
sp_nl2sql3 = sp_nl2sql3b.format(question="Return The name of the best paid employee")
print (sp_nl2sql3)

In [ ]:
input_sentences = tokenizer(sp_nl2sql3, return_tensors="pt").to('cuda')
response = get_outputs(foundation_model, input_sentences, max_new_tokens=400)
SQL = tokenizer.batch_decode(response, skip_special_tokens=True)
torch.cuda.empty_cache()

In [ ]:
print(SQL[0].split("```sql3")[-1].split("```")[0].split(";")[0].strip() + ";")

#Now the question in spanish.


In [ ]:
sp_nl2sql3 = sp_nl2sql3b.format(question="YOUR QUERY HERE")
print (sp_nl2sql3)

In [ ]:
input_sentences = tokenizer(sp_nl2sql3, return_tensors="pt").to('cuda')
response = get_outputs(foundation_model, input_sentences, max_new_tokens=400)
SQL = tokenizer.batch_decode(response, skip_special_tokens=True)
torch.cuda.empty_cache()

In [ ]:
print(SQL[0].split("```sql3")[-1].split("```")[0].split(";")[0].strip() + ";")

The generated SQL command is the same regardless of where we have placed the examples.

#Conclusions.

Let's see the three SQL's together.

* SELECT employees.name, MAX(salary.salary) AS max_salary FROM employees JOIN salary ON employees.ID_Usr = salary.ID_Usr GROUP BY employees.name ORDER BY max_salary DESC NULLS LAST LIMIT 1;

* SELECT e.name
    FROM employees e
    JOIN salary s ON e.ID_Usr = s.ID_usr
    WHERE s.salary = (SELECT MAX(salary) FROM salary);

* SELECT e.name
    FROM employees e
    JOIN salary s ON e.ID_Usr = s.ID_usr
    WHERE s.salary = (SELECT MAX(salary) FROM salary);

* Spanish Question: SELECT e.name
     FROM employees e
     JOIN salary s ON e.ID_Usr = s.ID_Usr
     WHERE s.salary = (SELECT MAX(salary) FROM salary)
     GROUP BY e.name
     ORDER BY COUNT(studies.ID_study) DESC
     LIMIT 1;


**The model has demonstrated that it is highly efficient in crafting SQL.** Additionally, it pays a lot of attention, perhaps too much, to the examples we provide. Clearly, these examples should be crafted by one of the best SQL programmers we have access to, though their use may not be essential.

On the other hand, although the model is clearly very proficient in SQL generation, during the creation of the notebook, I have encountered several issues because the commands need to be extremely clear. It doesn't handle typos well (which should not exist).

It appears to have some issues when it receives commands in Spanish. I assume this problem would be present in any language other than English. Therefore, since it's a tool that could be used by non-technical personnel, this should be considered in environments where English is not the primary language.

# Exercise
 - Complete the prompts similar to what we did in class.
     - Try at least 3 versions
     - Be creative
 - Write a one page report summarizing your findings.
     - Were there variations that didn't work well? i.e., where GPT either hallucinated or wrong
 - What did you learn?

 Experiment with Prompt Design
Create three different prompts to test the SQL generation model under varying scenarios. For each prompt:

Define a clear instruction.
Optionally, provide examples to guide the model.
Change specific aspects like language, complexity of the query, or database schema details.
Example Variations:
Prompt 1: Basic Schema and Simple Query
Provide a simple schema and ask for a basic SQL query.
Prompt 2: Schema with Ambiguity
Use a schema with potential ambiguities or missing details, testing the model's assumptions.
Prompt 3: Multilingual Prompt
Test instructions and questions in non-English languages (e.g., Spanish or French).
2. Run the Model
For each prompt:

Load and tokenize the input using the provided Colab setup.
Generate the SQL query using the model.
Analyze the output for accuracy, completeness, and adherence to the database schema.
3. Record Observations
Document:

Accuracy: Did the model generate correct SQL?
Clarity: Were the instructions understood as intended?
Style Consistency: Did the model follow the SQL style requested?
Language Sensitivity: Were non-English prompts handled correctly?
4. Write a Report
Summarize your findings in a one-page document.

Report Outline
Title:
Evaluation of SQL Generation Using a Fine-Tuned Mistral Model

Sections:
Introduction
Briefly describe the purpose of the experiment (evaluating SQL generation).

Experiment Design
Explain the variations in prompt design and why they were chosen.

Findings

Prompt 1 (Basic Schema): Highlight the model's performance in a straightforward scenario.
Prompt 2 (Ambiguous Schema): Discuss how the model handled incomplete instructions.
Prompt 3 (Multilingual): Address any challenges or successes with non-English inputs.
Challenges Faced

Handling ambiguous instructions.
Model sensitivity to typos and language inconsistencies.
Conclusions

The model excels when prompts are clear and detailed.
Examples enhance output quality but require precision.
Non-English inputs may need additional fine-tuning or translation support.
Recommendations

Ensure clear, typo-free prompts.
Provide high-quality examples.
For non-English queries, consider pre-translation or multilingual fine-tuning.